In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/booksdataset/Book_dataset.csv.xlsx


In [2]:
books=pd.read_excel('/kaggle/input/booksdataset/Book_dataset.csv.xlsx')
books.head()

,Wikipedia article ID,Freebase ID,Book title,Author,Publication date,Book genres (Freebase ID:name tuples),Plot summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


In [3]:
books.rename(columns={'Book genres (Freebase ID:name tuples)':'genres','Book title':'Book_title','Plot summary':'Plot_summary'},inplace=True)

In [4]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16559 entries, 0 to 16558
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Wikipedia article ID  16559 non-null  int64 
 1   Freebase ID           16559 non-null  object
 2   Book_title            16559 non-null  object
 3   Author                14177 non-null  object
 4   Publication date      10949 non-null  object
 5   genres                12841 non-null  object
 6   Plot_summary          16559 non-null  object
dtypes: int64(1), object(6)
memory usage: 905.7+ KB


In [5]:
books=books[['Wikipedia article ID','Freebase ID','Book_title','Author','genres','Plot_summary']]

In [6]:
books.dropna(inplace=True)

In [7]:
books.drop(books[books['genres']==''].index, inplace=True)
books[books['genres']=='']

,Wikipedia article ID,Freebase ID,Book_title,Author,genres,Plot_summary


In [8]:
json.loads(books['genres'][0]).values()

dict_values(['Roman à clef', 'Satire', "Children's literature", 'Speculative fiction', 'Fiction'])

In [9]:
genre = []
for i in books['genres']:
    genre.append(list(json.loads(i).values()))
books['genre_new'] = genre

In [10]:
books

,Wikipedia article ID,Freebase ID,Book_title,Author,genres,Plot_summary,genre_new
0,620,/m/0hhy,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca...","[Roman à clef, Satire, Children's literature, ..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan...","[Science Fiction, Novella, Speculative fiction..."
2,986,/m/0ldx,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...,"[Existentialism, Fiction, Absurdist fiction, N..."
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...,"[Hard science fiction, Science Fiction, Specul..."
5,2152,/m/0x5g,All Quiet on the Western Front,Erich Maria Remarque,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman...","The book tells the story of Paul BÃ¤umer, a G...","[War novel, Roman à clef]"
...,...,...,...,...,...,...,...
16549,36372465,/m/02vqwsp,The Third Lynx,Timothy Zahn,"{""/m/06n90"": ""Science Fiction""}",The story starts with former government agent...,[Science Fiction]
16551,36534061,/m/072y44,Remote Control,Andy McNab,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction...",The series follows the character of Nick Ston...,"[Thriller, Fiction, Suspense]"
16555,37054020,/m/04f1nbs,Transfer of Power,Vince Flynn,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction""}",The reader first meets Rapp while he is doing...,"[Thriller, Fiction]"
16556,37122323,/m/0n5236t,Decoded,Jay-Z,"{""/m/0xdf"": ""Autobiography""}",The book follows very rough chronological ord...,[Autobiography]


In [11]:
def clean_summary(text):
    text = re.sub("\'", "", text)
    text = re.sub("[^a-zA-Z]"," ",text)
    text = ' '.join(text.split())
    text = text.lower()
    return text

In [12]:
books['summary'] = books['Plot_summary'].apply(lambda x: clean_summary(x))
books.head(2)

,Wikipedia article ID,Freebase ID,Book_title,Author,genres,Plot_summary,genre_new,summary
0,620,/m/0hhy,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca...","[Roman à clef, Satire, Children's literature, ...",old major the old boar on the manor farm calls...
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan...","[Science Fiction, Novella, Speculative fiction...",alex a teenager living in near future england ...


In [13]:
books['summary'][0]

'old major the old boar on the manor farm calls the animals on the farm for a meeting where he compares the humans to parasites and teaches the animals a revolutionary song beasts of england when major dies two young pigs snowball and napoleon assume command and turn his dream into a philosophy the animals revolt and drive the drunken and irresponsible mr jones from the farm renaming it animal farm they adopt seven commandments of animal ism the most important of which is all animals are equal snowball attempts to teach the animals reading and writing food is plentiful and the farm runs smoothly the pigs elevate themselves to positions of leadership and set aside special food items ostensibly for their personal health napoleon takes the pups from the farm dogs and trains them privately napoleon and snowball struggle for leadership when snowball announces his plans to build a windmill napoleon has his dogs chase snowball away and declares himself leader napoleon enacts changes to the go

In [14]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    no_stopword_text = [i for i in text.split() if not i in stop_words]
    return ' '.join(no_stopword_text)

books['summary'] = books['summary'].apply(lambda x: remove_stopwords(x))

In [15]:
books

,Wikipedia article ID,Freebase ID,Book_title,Author,genres,Plot_summary,genre_new,summary
0,620,/m/0hhy,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca...","[Roman à clef, Satire, Children's literature, ...",old major old boar manor farm calls animals fa...
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan...","[Science Fiction, Novella, Speculative fiction...",alex teenager living near future england leads...
2,986,/m/0ldx,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...,"[Existentialism, Fiction, Absurdist fiction, N...",text plague divided five parts town oran thous...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...,"[Hard science fiction, Science Fiction, Specul...",novel posits space around milky way divided co...
5,2152,/m/0x5g,All Quiet on the Western Front,Erich Maria Remarque,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman...","The book tells the story of Paul BÃ¤umer, a G...","[War novel, Roman à clef]",book tells story paul b umer german soldier ur...
...,...,...,...,...,...,...,...,...
16549,36372465,/m/02vqwsp,The Third Lynx,Timothy Zahn,"{""/m/06n90"": ""Science Fiction""}",The story starts with former government agent...,[Science Fiction],story starts former government agent frank com...
16551,36534061,/m/072y44,Remote Control,Andy McNab,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction...",The series follows the character of Nick Ston...,"[Thriller, Fiction, Suspense]",series follows character nick stone ex militar...
16555,37054020,/m/04f1nbs,Transfer of Power,Vince Flynn,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction""}",The reader first meets Rapp while he is doing...,"[Thriller, Fiction]",reader first meets rapp covert operation iran ...
16556,37122323,/m/0n5236t,Decoded,Jay-Z,"{""/m/0xdf"": ""Autobiography""}",The book follows very rough chronological ord...,[Autobiography],book follows rough chronological order switchi...


In [16]:
books['Author_']=books['Author'].str.replace(" ","")

In [17]:
books

,Wikipedia article ID,Freebase ID,Book_title,Author,genres,Plot_summary,genre_new,summary,Author_
0,620,/m/0hhy,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca...","[Roman à clef, Satire, Children's literature, ...",old major old boar manor farm calls animals fa...,GeorgeOrwell
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan...","[Science Fiction, Novella, Speculative fiction...",alex teenager living near future england leads...,AnthonyBurgess
2,986,/m/0ldx,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...,"[Existentialism, Fiction, Absurdist fiction, N...",text plague divided five parts town oran thous...,AlbertCamus
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...,"[Hard science fiction, Science Fiction, Specul...",novel posits space around milky way divided co...,VernorVinge
5,2152,/m/0x5g,All Quiet on the Western Front,Erich Maria Remarque,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman...","The book tells the story of Paul BÃ¤umer, a G...","[War novel, Roman à clef]",book tells story paul b umer german soldier ur...,ErichMariaRemarque
...,...,...,...,...,...,...,...,...,...
16549,36372465,/m/02vqwsp,The Third Lynx,Timothy Zahn,"{""/m/06n90"": ""Science Fiction""}",The story starts with former government agent...,[Science Fiction],story starts former government agent frank com...,TimothyZahn
16551,36534061,/m/072y44,Remote Control,Andy McNab,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction...",The series follows the character of Nick Ston...,"[Thriller, Fiction, Suspense]",series follows character nick stone ex militar...,AndyMcNab
16555,37054020,/m/04f1nbs,Transfer of Power,Vince Flynn,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction""}",The reader first meets Rapp while he is doing...,"[Thriller, Fiction]",reader first meets rapp covert operation iran ...,VinceFlynn
16556,37122323,/m/0n5236t,Decoded,Jay-Z,"{""/m/0xdf"": ""Autobiography""}",The book follows very rough chronological ord...,[Autobiography],book follows rough chronological order switchi...,Jay-Z


In [18]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [19]:
books['genre_new_'] = books['genre_new'].apply(collapse)


In [20]:
books.head(5)

,Wikipedia article ID,Freebase ID,Book_title,Author,genres,Plot_summary,genre_new,summary,Author_,genre_new_
0,620,/m/0hhy,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca...","[Roman à clef, Satire, Children's literature, ...",old major old boar manor farm calls animals fa...,GeorgeOrwell,"[Romanàclef, Satire, Children'sliterature, Spe..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan...","[Science Fiction, Novella, Speculative fiction...",alex teenager living near future england leads...,AnthonyBurgess,"[ScienceFiction, Novella, Speculativefiction, ..."
2,986,/m/0ldx,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...,"[Existentialism, Fiction, Absurdist fiction, N...",text plague divided five parts town oran thous...,AlbertCamus,"[Existentialism, Fiction, Absurdistfiction, No..."
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...,"[Hard science fiction, Science Fiction, Specul...",novel posits space around milky way divided co...,VernorVinge,"[Hardsciencefiction, ScienceFiction, Speculati..."
5,2152,/m/0x5g,All Quiet on the Western Front,Erich Maria Remarque,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman...","The book tells the story of Paul BÃ¤umer, a G...","[War novel, Roman à clef]",book tells story paul b umer german soldier ur...,ErichMariaRemarque,"[Warnovel, Romanàclef]"


In [21]:
books['Author_'] = books['Author_'].apply(lambda x:x.split())


In [22]:
books['summary'] = books['summary'].apply(lambda x:x.split())


In [23]:
books['tags']=books['summary']+books['Author_']+books['genre_new_']

In [24]:
books['tags'][0]

['old',
 'major',
 'old',
 'boar',
 'manor',
 'farm',
 'calls',
 'animals',
 'farm',
 'meeting',
 'compares',
 'humans',
 'parasites',
 'teaches',
 'animals',
 'revolutionary',
 'song',
 'beasts',
 'england',
 'major',
 'dies',
 'two',
 'young',
 'pigs',
 'snowball',
 'napoleon',
 'assume',
 'command',
 'turn',
 'dream',
 'philosophy',
 'animals',
 'revolt',
 'drive',
 'drunken',
 'irresponsible',
 'mr',
 'jones',
 'farm',
 'renaming',
 'animal',
 'farm',
 'adopt',
 'seven',
 'commandments',
 'animal',
 'ism',
 'important',
 'animals',
 'equal',
 'snowball',
 'attempts',
 'teach',
 'animals',
 'reading',
 'writing',
 'food',
 'plentiful',
 'farm',
 'runs',
 'smoothly',
 'pigs',
 'elevate',
 'positions',
 'leadership',
 'set',
 'aside',
 'special',
 'food',
 'items',
 'ostensibly',
 'personal',
 'health',
 'napoleon',
 'takes',
 'pups',
 'farm',
 'dogs',
 'trains',
 'privately',
 'napoleon',
 'snowball',
 'struggle',
 'leadership',
 'snowball',
 'announces',
 'plans',
 'build',
 'windmi

In [25]:
new_df = books.drop(columns=['Wikipedia article ID','Freebase ID','genres'])

In [26]:
new_df.head(2)

,Book_title,Author,Plot_summary,genre_new,summary,Author_,genre_new_,tags
0,Animal Farm,George Orwell,"Old Major, the old boar on the Manor Farm, ca...","[Roman à clef, Satire, Children's literature, ...","[old, major, old, boar, manor, farm, calls, an...",[GeorgeOrwell],"[Romanàclef, Satire, Children'sliterature, Spe...","[old, major, old, boar, manor, farm, calls, an..."
1,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...","[Science Fiction, Novella, Speculative fiction...","[alex, teenager, living, near, future, england...",[AnthonyBurgess],"[ScienceFiction, Novella, Speculativefiction, ...","[alex, teenager, living, near, future, england..."


In [27]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
new_df.head()

,Book_title,Author,Plot_summary,genre_new,summary,Author_,genre_new_,tags
0,Animal Farm,George Orwell,"Old Major, the old boar on the Manor Farm, ca...","[Roman à clef, Satire, Children's literature, ...","[old, major, old, boar, manor, farm, calls, an...",[GeorgeOrwell],"[Romanàclef, Satire, Children'sliterature, Spe...",old major old boar manor farm calls animals fa...
1,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...","[Science Fiction, Novella, Speculative fiction...","[alex, teenager, living, near, future, england...",[AnthonyBurgess],"[ScienceFiction, Novella, Speculativefiction, ...",alex teenager living near future england leads...
2,The Plague,Albert Camus,The text of The Plague is divided into five p...,"[Existentialism, Fiction, Absurdist fiction, N...","[text, plague, divided, five, parts, town, ora...",[AlbertCamus],"[Existentialism, Fiction, Absurdistfiction, No...",text plague divided five parts town oran thous...
4,A Fire Upon the Deep,Vernor Vinge,The novel posits that space around the Milky ...,"[Hard science fiction, Science Fiction, Specul...","[novel, posits, space, around, milky, way, div...",[VernorVinge],"[Hardsciencefiction, ScienceFiction, Speculati...",novel posits space around milky way divided co...
5,All Quiet on the Western Front,Erich Maria Remarque,"The book tells the story of Paul BÃ¤umer, a G...","[War novel, Roman à clef]","[book, tells, story, paul, b, umer, german, so...",[ErichMariaRemarque],"[Warnovel, Romanàclef]",book tells story paul b umer german soldier ur...


In [28]:
new_df['tags'][5]

'book tells story paul b umer german soldier urged school teacher joins german army shortly start world war b umer arrives western front friends schoolmates tjaden ller kropp number characters meet stanislaus katczinsky older soldier nicknamed kat becomes pauls mentor fighting front b umer comrades engage frequent battles endure dangerous often dirty conditions warfare beginning book erich maria remarque says book neither accusation confession least adventure death adventure stand face face try simply tell generation men even though may escaped shells destroyed war book focus heroic stories bravery rather gives view conditions soldiers find monotony battles constant threat artillery fire bombardments struggle find food lack training young recruits meaning lower chances survival overarching role random chance lives deaths soldiers described detail battles fought names seem little overall significance except impending possibility injury death b umer comrades pitifully small pieces land g

In [29]:
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()

In [30]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [31]:
new_df['tags']=new_df['tags'].apply(stem)

In [32]:
new_df['tags'][0]

"old major old boar manor farm call anim farm meet compar human parasit teach anim revolutionari song beast england major die two young pig snowbal napoleon assum command turn dream philosophi anim revolt drive drunken irrespons mr jone farm renam anim farm adopt seven command anim ism import anim equal snowbal attempt teach anim read write food plenti farm run smoothli pig elev posit leadership set asid special food item ostens person health napoleon take pup farm dog train privat napoleon snowbal struggl leadership snowbal announc plan build windmil napoleon dog chase snowbal away declar leader napoleon enact chang govern structur farm replac meet committe pig run farm use young pig name squealer mouthpiec napoleon claim credit windmil idea anim work harder promis easier live windmil violent storm anim find windmil annihil napoleon squealer convinc anim snowbal destroy although scorn neighbour farmer suggest wall thin snowbal becom scapegoat napoleon begin purg farm dog kill anim acc

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000)

In [34]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [35]:
cv.get_feature_names_out()

array(['aaron', 'abandon', 'abbey', ..., 'zombi', 'zone', 'zoo'],
      dtype=object)

In [36]:
from sklearn.metrics.pairwise import cosine_similarity

In [37]:
similarity=cosine_similarity(vectors)

In [38]:
similarity[0]

array([1.        , 0.05801463, 0.0675753 , ..., 0.04235067, 0.02878608,
       0.07442623])

In [39]:
new_df.reset_index(inplace=True)

In [40]:
new_df

,index,Book_title,Author,Plot_summary,genre_new,summary,Author_,genre_new_,tags
0,0,Animal Farm,George Orwell,"Old Major, the old boar on the Manor Farm, ca...","[Roman à clef, Satire, Children's literature, ...","[old, major, old, boar, manor, farm, calls, an...",[GeorgeOrwell],"[Romanàclef, Satire, Children'sliterature, Spe...",old major old boar manor farm call anim farm m...
1,1,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...","[Science Fiction, Novella, Speculative fiction...","[alex, teenager, living, near, future, england...",[AnthonyBurgess],"[ScienceFiction, Novella, Speculativefiction, ...",alex teenag live near futur england lead gang ...
2,2,The Plague,Albert Camus,The text of The Plague is divided into five p...,"[Existentialism, Fiction, Absurdist fiction, N...","[text, plague, divided, five, parts, town, ora...",[AlbertCamus],"[Existentialism, Fiction, Absurdistfiction, No...",text plagu divid five part town oran thousand ...
3,4,A Fire Upon the Deep,Vernor Vinge,The novel posits that space around the Milky ...,"[Hard science fiction, Science Fiction, Specul...","[novel, posits, space, around, milky, way, div...",[VernorVinge],"[Hardsciencefiction, ScienceFiction, Speculati...",novel posit space around milki way divid conce...
4,5,All Quiet on the Western Front,Erich Maria Remarque,"The book tells the story of Paul BÃ¤umer, a G...","[War novel, Roman à clef]","[book, tells, story, paul, b, umer, german, so...",[ErichMariaRemarque],"[Warnovel, Romanàclef]",book tell stori paul b umer german soldier urg...
...,...,...,...,...,...,...,...,...,...
12050,16549,The Third Lynx,Timothy Zahn,The story starts with former government agent...,[Science Fiction],"[story, starts, former, government, agent, fra...",[TimothyZahn],[ScienceFiction],stori start former govern agent frank compton ...
12051,16551,Remote Control,Andy McNab,The series follows the character of Nick Ston...,"[Thriller, Fiction, Suspense]","[series, follows, character, nick, stone, ex, ...",[AndyMcNab],"[Thriller, Fiction, Suspense]",seri follow charact nick stone ex militari man...
12052,16555,Transfer of Power,Vince Flynn,The reader first meets Rapp while he is doing...,"[Thriller, Fiction]","[reader, first, meets, rapp, covert, operation...",[VinceFlynn],"[Thriller, Fiction]",reader first meet rapp covert oper iran discov...
12053,16556,Decoded,Jay-Z,The book follows very rough chronological ord...,[Autobiography],"[book, follows, rough, chronological, order, s...",[Jay-Z],[Autobiography],book follow rough chronolog order switch curre...


In [41]:
def recommend(book):
    book_index = new_df[new_df['Book_title']== book].index[0]
    distances = similarity[book_index]
    book_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in book_list:
        print('Book-title:','     ',new_df.iloc[i[0]].Book_title)
        print()
        print('Author-Name:','    ',new_df.iloc[i[0]].Author)
        print()
        print('Genre:','          ',new_df.iloc[i[0]].genre_new)
        print()
        print('Book-Summary:','   ',new_df.iloc[i[0]].Plot_summary)
        print()

In [42]:
new_df['Book_title'].tail(50)

12005                              Mr. Monk is Cleaned Out
12006                                 Mr. Monk on the Road
12007                                Mr. Monk on the Couch
12008                                   Mr. Monk on Patrol
12009                                   The Sands of Ammon
12010                                The Ends of the Earth
12011                                              Spartan
12012    The Fiery Trial: Abraham Lincoln and American ...
12013                                      The Shield Ring
12014                          Of Wee Sweetie Mice and Men
12015                                    Turbulent Priests
12016                                   She Lover of Death
12017                                        Shooting Sean
12018                               The Horse With My Name
12019                                     The Summer Birds
12020                                       The Black Rose
12021                              Big Nate: Strikes Aga

In [43]:
recommend('Dune')

Book-title:       Paul of Dune

Author-Name:      Kevin J. Anderson

Genre:            ['Science Fiction', 'Speculative fiction']

Book-Summary:      The book is divided into seven sections, which alternate between Paul Atreides's youth before the events portrayed in Dune, and the early period of his Fremen jihad between Dune and Dune Messiah. Twelve-year-old Paul resides on the planet Caladan with his parents, Duke Leto Atreides and his Bene Gesserit concubine Lady Jessica. House Ecaz of Ecaz and House Moritani of Grumman are embroiled in a generations-long feud, and an Atreides-Ecazi alliance is set to be formalized by Leto's marriage to the Archduke Armand Ecaz's daughter Illesa. At the wedding, Leto and his family escape an assassination attempt, but Armand is injured and Illesa is killed. Leto and Armand lead a retaliatory attack on Grumman, not realizing that the Moritani forces have been supplemented by troops from House Harkonnen, sworn enemies of the Atreides. The Padishah Emp